In [9]:
import os
from dotenv import load_dotenv
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import OpenAI
from langchain_community.utilities import SerpAPIWrapper

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
GOOGLE_CSE_ID = os.getenv('GOOGLE_CSE_ID')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Tool 1: 用 SerpAPI 搜尋
serpapi_search = SerpAPIWrapper()
serpapi_tool = Tool(
    name="SerpAPI Search",
    func=serpapi_search.run,
    description="使用 SerpAPI 查詢 Google 搜尋結果"
)

# Tool 2: 用 Google CSE API 搜尋
def google_cse_search(query: str, **kwargs):
    try:
        from googleapiclient.discovery import build
        service = build(
            serviceName="customsearch",
            version="v1",
            developerKey=GOOGLE_API_KEY,
            static_discovery=False
        )
        res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, **kwargs).execute()
        items = res.get("items", [])
        snippets = [item.get("snippet", "") for item in items]
        return "\n".join(snippets[:3]) if snippets else "none"
    except Exception as e:
        return f"Google CSE error: {str(e)}"

cse_tool = Tool(
    name="Google CSE Search",
    func=google_cse_search,
    description="使用 Google 自訂搜尋 API"
)

llm = OpenAI(temperature=0)

# 整合兩個搜尋工具給 Agent 用
tools = [serpapi_tool, cse_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 測試
article_content = """現在(2025/3/27)的輝達股價是180"""
response = agent.run(f"Please go to external websites to help me look up the related information. Article: \"{article_content}\". Based on your analysis, and all the information you search. Does this article contain any misleading information or mismatched relationships? Show your analytic process and If this article is more likely to contain misleading information, return 1; otherwise, return 0. The answer (Arabic numerals) is:")
print("📌 Answer：", response)




> Entering new AgentExecutor chain...
 I should use a search engine to look up the current stock price of Nvidia.
Action: SerpAPI Search
Action Input: "Nvidia stock price"
Observation: {'type': 'finance_results', 'title': 'NVIDIA Corp', 'exchange': 'NASDAQ', 'stock': ' NVDA', 'currency': 'USD', 'price': 114.06, 'previous_close': 120.69}
Thought: This is the current stock price of Nvidia, but it is not the date mentioned in the article. I should try to find the stock price on the date mentioned in the article.
Action: Google CSE Search
Action Input: "Nvidia stock price 2025/3/27"
Observation: Unlock stock picks and a broker-level newsfeed that powers Wall Street. Upgrade Now. NasdaqGS - Delayed Quote • USD. NVIDIA Corporation (NVDA). Follow. Compare.
Updated March 25, 2025, 3:27 pm EDT / Original March 25, 2025, 11:45 am EDT ... Nvidia stock is down 1.1% at $120.20. Advertisement - Scroll to Continue.
Mar 15, 2020 ... Premarket Thread for General Trading and Plans for Thursday, March 